# Step-by-step understanding LSTM Autoencoder layers

Here we will break down an LSTM autoencoder network to understand them layer-by-layer. We will go over the input and output flow between the layers, and also, compare the LSTM Autoencoder with a regular LSTM network.

ULR: https://towardsdatascience.com/step-by-step-understanding-lstm-autoencoder-layers-ffab055b6352

In [9]:
# lstm autoencoder to recreate a timeseries
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed

In [10]:
'''
A UDF to convert input data into 3-D
array as required for LSTM network.
'''

def temporalize(X, y, lookback):
    output_X = []
    output_y = []
    for i in range(len(X)-lookback-1):
        t = []
        for j in range(1,lookback+1):
            # Gather past records upto the lookback period
            t.append(X[[(i+j+1)], :])
        output_X.append(t)
        output_y.append(y[i+lookback+1])
    return output_X, output_y

In [11]:
# define input timeseries
timeseries = np.array([[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
                       [0.1**3, 0.2**3, 0.3**3, 0.4**3, 0.5**3, 0.6**3, 0.7**3, 0.8**3, 0.9**3]]).transpose()

timesteps = timeseries.shape[0]
n_features = timeseries.shape[1]
print(timesteps, n_features)
timeseries

9 2


array([[0.1  , 0.001],
       [0.2  , 0.008],
       [0.3  , 0.027],
       [0.4  , 0.064],
       [0.5  , 0.125],
       [0.6  , 0.216],
       [0.7  , 0.343],
       [0.8  , 0.512],
       [0.9  , 0.729]])

In [12]:
timesteps = 3
X, y = temporalize(X = timeseries, y = np.zeros(len(timeseries)), lookback = timesteps)

n_features = 2
X = np.array(X)
X = X.reshape(X.shape[0], timesteps, n_features)

print(X.shape)
print(len(y))
X

(5, 3, 2)
5


array([[[0.3  , 0.027],
        [0.4  , 0.064],
        [0.5  , 0.125]],

       [[0.4  , 0.064],
        [0.5  , 0.125],
        [0.6  , 0.216]],

       [[0.5  , 0.125],
        [0.6  , 0.216],
        [0.7  , 0.343]],

       [[0.6  , 0.216],
        [0.7  , 0.343],
        [0.8  , 0.512]],

       [[0.7  , 0.343],
        [0.8  , 0.512],
        [0.9  , 0.729]]])

In [13]:
# define model
model = Sequential()
model.add(LSTM(128, activation='relu', input_shape=(timesteps,n_features), return_sequences=True))
model.add(LSTM(64, activation='relu', return_sequences=False))
model.add(RepeatVector(timesteps))
model.add(LSTM(64, activation='relu', return_sequences=True))
model.add(LSTM(128, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(n_features)))
model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 3, 128)            67072     
_________________________________________________________________
lstm_5 (LSTM)                (None, 64)                49408     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 3, 64)             0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 3, 64)             33024     
_________________________________________________________________
lstm_7 (LSTM)                (None, 3, 128)            98816     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 3, 2)              258       
Total params: 248,578
Trainable params: 248,578
Non-trainable params: 0
________________________________________________

In [14]:
# fit model
model.fit(X, X, epochs=300, batch_size=5, verbose=1)

Epoch 1/300
1/1 [==============================] - 3s 3s/step - loss: 0.2468
Epoch 2/300
1/1 [==============================] - 0s 5ms/step - loss: 0.2442
Epoch 3/300
1/1 [==============================] - 0s 5ms/step - loss: 0.2418
Epoch 4/300
1/1 [==============================] - 0s 6ms/step - loss: 0.2392
Epoch 5/300
1/1 [==============================] - 0s 5ms/step - loss: 0.2366
Epoch 6/300
1/1 [==============================] - 0s 5ms/step - loss: 0.2337
Epoch 7/300
1/1 [==============================] - 0s 6ms/step - loss: 0.2308
Epoch 8/300
1/1 [==============================] - 0s 5ms/step - loss: 0.2276
Epoch 9/300
1/1 [==============================] - 0s 5ms/step - loss: 0.2243
Epoch 10/300
1/1 [==============================] - 0s 6ms/step - loss: 0.2208
Epoch 11/300
1/1 [==============================] - 0s 5ms/step - loss: 0.2170
Epoch 12/300
1/1 [==============================] - 0s 7ms/step - loss: 0.2130
Epoch 13/300
1/1 [==============================] - 0s 5ms/ste

In [15]:
# demonstrate reconstruction
yhat = model.predict(X, verbose=0)
print('---Predicted---')
print(np.round(yhat,3))
print('---Actual---')
print(np.round(X, 3))

---Predicted---
[[[0.339 0.043]
  [0.439 0.081]
  [0.507 0.137]]

 [[0.395 0.065]
  [0.503 0.13 ]
  [0.586 0.214]]

 [[0.476 0.107]
  [0.588 0.213]
  [0.694 0.339]]

 [[0.583 0.184]
  [0.689 0.338]
  [0.814 0.509]]

 [[0.712 0.36 ]
  [0.802 0.515]
  [0.895 0.727]]]
---Actual---
[[[0.3   0.027]
  [0.4   0.064]
  [0.5   0.125]]

 [[0.4   0.064]
  [0.5   0.125]
  [0.6   0.216]]

 [[0.5   0.125]
  [0.6   0.216]
  [0.7   0.343]]

 [[0.6   0.216]
  [0.7   0.343]
  [0.8   0.512]]

 [[0.7   0.343]
  [0.8   0.512]
  [0.9   0.729]]]
